In [2]:
import torch
from LabData.DataLoaders.PRSLoader import PRSLoader
import torch.nn as nn
from torch.utils.checkpoint import checkpoint_sequential
import torch.nn.functional as F
import torch.optim as optim
from genetics_dataset import GeneticsDataset

data = GeneticsDataset()

Mapping files: 100%|██████████████████████████████| 3/3 [00:01<00:00,  1.72it/s]


In [2]:
class fc_snpnet(nn.Module):
    def __init__(self):
        super(fc_snpnet, self).__init__()
        self.conv1 = nn.Conv1d(31, 31, 2000)
        self.conv2 = nn.Conv1d(31, 31, 1000)
        self.conv4 = nn.Conv1d(31, 31, 1000)
        self.conv6 = nn.Conv1d(31, 31, 1000)
        self.conv7 = nn.Conv1d(31, 31, 1000)
        self.fc1 = nn.Linear(3596, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 516)
        self.fc4 = nn.Linear(516, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.reshape(x, [2, 31, 6053])
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv4(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = x.flatten()
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        x = self.fc6(x)
        return x

In [3]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape
        
    def forward(self, x):
        return x.view(*self.shape)

net = nn.Sequential(
        Reshape([2, 31, 6053]),
        nn.Conv1d(31, 31, 2000),
        nn.Conv1d(31, 31, 1000),
        nn.Conv1d(31, 31, 1000),
        nn.Conv1d(31, 31, 1000),
        nn.Conv1d(31, 31, 1000),
        nn.Flatten(),
        nn.Linear(3596, 2048),
        nn.Sigmoid(),
        nn.Linear(2048, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, 516),
        nn.Sigmoid(),
        nn.Linear(516, 256),
        nn.Sigmoid(),
        nn.Linear(256, 128),
        nn.Sigmoid(),
        nn.Linear(128, 1)
)

#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
net = net.to(device)

In [8]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.2)
for epoch in range(30):
    i = 0
    avg_loss = 0
    for tenK_index in list(data.binaries.sample.values)[0:2]:
        try:
            snps, label = data.__getitem__(tenK_index)
            snps = snps.to(device).requires_grad_()
            label = label.to(device)
            optimizer.zero_grad()
            outputs = checkpoint_sequential(net, 1, snps)
            loss = criterion(outputs, label)
            avg_loss += float(loss)
            loss.sum().backward()
            optimizer.step()
        except IndexError:
            pass
        i += 1
    print("Epoch: " + str(epoch) + " , avg loss = " + str(avg_loss/i))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x1798 and 3596x2048)